In [1]:
import pandas as pd

In [2]:
juegos = pd.read_csv('games_final.csv')
items = pd.read_csv('items_final_pulido.csv')
reviews = pd.read_csv('reviews_final.csv')

In [3]:
juegos = juegos.dropna(subset=['genres']) #elimino los juegos que no tienen género ya que no me sirven

In [4]:
items.head(1)

,user_id,item_id,item_name,playtime_forever
0,phrostb,300,Day of Defeat: Source,78


In [4]:
reviews.posted = pd.to_datetime(reviews.posted).dt.year

In [5]:
reviews_fecha = reviews.drop(columns=['funny', 'helpful', 'recommend', 'review'])

In [6]:
juegos_años = pd.merge(items, reviews_fecha, on= ['user_id', 'item_id'], how='inner')

In [14]:
print(items.shape)
print(juegos_años.shape)

(303631, 4)
(3072, 5)


In [12]:
juegos_años.head(10)

,user_id,item_id,item_name,playtime_forever,posted
0,phrostb,70,Half-Life,1303,2014-01-19
1,phrostb,620,Portal 2,1370,2014-03-25
2,phrostb,45760,Ultra Street Fighter IV,1752,2013-11-01
3,phrostb,238210,System Shock 2,1574,2014-03-23
4,phrostb,207750,Symphony,42,2014-03-30
5,phrostb,300040,Divine Souls,16,2014-11-23
6,phrostb,387240,Steel Rain,4490,2015-12-23
7,thugnificent,321030,One Day For Ched,8,2014-09-28
8,chidvd,400,Portal,488,2010-12-20
9,chidvd,220,Half-Life 2,197,2012-02-01


In [7]:
#paso la columna posted a datetime y me quedo con los años
juegos_años.posted = pd.to_datetime(juegos_años.posted).dt.year

In [8]:
#la columna playtime_forever está en minutos, la paso a años.
juegos_años.playtime_forever = juegos_años.playtime_forever/60

In [9]:
def UserForGenre(genero):
    #obtengo liste de ids de juegos que se corresponden con ese género
    juegos_id = juegos[juegos.genres.apply(lambda x:(genero.capitalize() in x) )]['id']
    #creo diccionario con jugadores y horas jugadas
    jugadores = juegos_años.user_id.unique()
    horas_jugadas = { jugador:0 for jugador in jugadores}
   #filtro el df con los ids de los juegos que responden al género 
    juegos_filtrados = juegos_años[juegos_años['item_id'].isin(juegos_id)]
    # itero para sumar horas jugadas para cada jugador
    for indice, fila in juegos_filtrados.iterrows():
        horas_jugadas[fila['user_id']] += fila['playtime_forever']
    #jugador con más horas jugadas
    max_user = max(horas_jugadas, key=horas_jugadas.get)
    # diccionario para calcular años con más horas jugadas.
    horas_por_año = {año:0 for año in juegos_años.posted.unique()}

    juegos_filtrados_por_usuario = juegos_años[(juegos_años['item_id'].isin(juegos_id)) & (juegos_años['user_id'] == max_user)]

    for indice, fila in juegos_filtrados_por_usuario.iterrows():
        horas_por_año[fila['posted']] += fila['playtime_forever']

    
    horas_por_año_ordenado = sorted(horas_por_año.items(), key=lambda x: x[1], reverse=True)
    
    horas_jugadas_top = [{"Año": año, "Horas": horas} for año, horas in horas_por_año_ordenado[:3] if horas > 0]
    
    return (f"Usuario con más horas jugadas para Género {genero.capitalize()}: {max_user}, Horas jugadas: {horas_jugadas_top}")

In [10]:
UserForGenre('ACTION')

"Usuario con más horas jugadas para Género Action: shinomegami, Horas jugadas: [{'Año': 1970, 'Horas': 11271.716666666667}]"

In [11]:
juegos.release_date = pd.to_datetime(juegos.release_date).dt.year

In [85]:
len(juegos.release_date.unique())

39

In [86]:
juegos.head(1)

,genres,app_name,title,release_date,tags,id
0,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,Lost Summoner Kitty,2018.0,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",761140.0


In [94]:
filtro = juegos['id'] == 761140.0
año_lanzamiento = int(juegos.loc[filtro, 'release_date'])
año_lanzamiento

C:\Users\torin\AppData\Local\Temp\ipykernel_10216\898201801.py:2: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  año_lanzamiento = int(juegos.loc[filtro, 'release_date'])


2018

In [12]:
def PlayTimeGenre(genero):
    #obtengo liste de ids de juegos que se corresponden con ese género
    juegos_id = juegos[juegos.genres.apply(lambda x:(genero.capitalize() in x) )]['id']
    #filtro el df con los ids de los juegos que responden al género 
    juegos_filtrados = juegos_años[juegos_años['item_id'].isin(juegos_id)]
    # itero para sumar horas jugadas para cada jugador
    horas_por_release_year = {year:0 for year in juegos.release_date.unique()}
    for indice, fila in juegos_filtrados.iterrows():
        filtro = juegos['id'] == fila['item_id']
        año_lanzamiento = int(juegos.loc[filtro, 'release_date'])
        horas_por_release_year[año_lanzamiento]+= fila['playtime_forever']
    
    año = max(horas_por_release_year, key=horas_por_release_year.get)

    return f"Año de lanzamiento con más horas jugadas para Género {genero.capitalize()} : {año}"
    
    

In [13]:
PlayTimeGenre('CaSuAl')

C:\Users\torin\AppData\Local\Temp\ipykernel_19840\1508120866.py:10: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  año_lanzamiento = int(juegos.loc[filtro, 'release_date'])


'Año de lanzamiento con más horas jugadas para Género Casual : 2011.0'

In [81]:
def UsersRecommend(año):
    if año not in reviews.posted.unique():
        return f"No hay recomendaciones para ese año. Pruebe con uno de estos años: {reviews.posted.unique()}"
    
    df = reviews[(reviews.posted == año)&(reviews['item_id'].isin(juegos.id))][['posted', 'item_id', 'recommend']]

    juegos_recomendados = {item :0 for item in df.item_id.unique()}
    
    for indice, fila in df.iterrows():
        juegos_recomendados[fila['item_id']] +=1
    
    juegos_recomendados = sorted(juegos_recomendados.items(), key=lambda x: x[1], reverse=True)
    
    juego1 = str(juegos[juegos['id'] == juegos_recomendados[0][0]]['title'].iloc[0])
    juego2 = str(juegos[juegos['id'] == juegos_recomendados[1][0]]['title'].iloc[0])
    juego3 = str(juegos[juegos['id'] == juegos_recomendados[2][0]]['title'].iloc[0])

    return f"Puesto 1 : {juego1}, Puesto 2 : {juego2}, Puesto 3 : {juego3}"
    

In [83]:
UsersRecommend(2014)

"Puesto 1 : Team Fortress 2, Puesto 2 : Counter-Strike: Global Offensive, Puesto 3 : Garry's Mod"

In [80]:
juegos[juegos['id'] ==550]

,genres,app_name,title,release_date,tags,id
31529,['Action'],Left 4 Dead 2,Left 4 Dead 2,2009.0,"['Zombies', 'Co-op', 'FPS', 'Multiplayer', 'Ac...",550.0
